In [1]:
import os, json
import numpy as np
import pandas as pd
from dLoader import DataLoader
from dLoader.utils import regroup

# Get Tickers from Database

In [52]:
directory = os.path.join(os.getcwd(), 'Database')
# Search Param
min_dollar = 0
max_dollar = 10
min_length = 250 * 6
today = pd.Timestamp.now()
# Placeholder
tickers = []
files = os.listdir(directory)
if 'keeper.json' in files:
    path = os.path.join(directory, 'keeper.json')
    db = pd.read_json(path, orient='index')
    # Get pricing
    mask = (db['Latest Price'].between(min_dollar, max_dollar)) & (db['Length in Database'] > min_length)
    ndb = db.loc[mask].copy()
    need_update = (today.day - pd.to_datetime(ndb['Last Updated']).dt.day) > 5
    if need_update.sum() > 0:
        jFile = json.load(open(path, 'r'))
        for ticker in ndb.loc[need_update].index.to_list():
            DataLoader(ticker).update_database()
            jFile[ticker]['Last Updated'] = today.strftime('%Y-%m-%d')
        # Write json file
        json.dump(jFile, open(path, 'w'), indent=4)
    tickers = db.loc[mask].index.to_list()
else:
    jFile = {}
    for file in files:
        name, ext = file.split('.')
        if ext == 'csv':
            df = DataLoader(name).data
            latest_price = df['Close'].iloc[-1]
            jFile[name] = {'Latest Price': latest_price,
                           'Length in Database': len(df),
                           'Last Updated': df.index[-1].strftime('%Y-%m-%d')}
            if len(df) > min_length and (min_dollar < latest_price < max_dollar):
                tickers.append(name)
    # Write json file
    with open(os.path.join(directory, 'keeper.json'), 'w') as outfile:
        json.dump(jFile, outfile, indent=4)

print('Total tickers met parameters in Database: {}'.format(len(tickers)))

Total tickers met parameters in Database: 253


In [53]:
def sim(data, sell_limit=.03, buy_limit=.03, max_hold=10):
    holding = False
    buy_at = 0
    cap = {}
    held = 0
    for d, prices in data.iterrows():
        if pd.isna(prices.Previous_Close):
            continue
        if not holding:
            check_low = prices.Low / prices.Previous_Close - 1
            if check_low < -buy_limit:
                buy_at = prices.Low * (1 + buy_limit)
                holding = True
        else:
            held += 1
            check_open = prices.Open / buy_at - 1
            check_high = prices.High / buy_at - 1
            check_close = prices.Close / buy_at - 1
            if check_open > sell_limit:
                cap[d] = {'Gain': check_open,
                          'Held': held}
                holding = False
                buy_at = 0
                held = 0
            elif check_high > sell_limit:
                # Trailling Sell
                cap[d] = {'Gain': sell_limit,
                          'Held': held}
                holding = False
                buy_at = 0
                held = 0
            elif held > max_hold:
                cap[d] = {'Gain': check_close,
                          'Held': held}
                holding = False
                buy_at = 0
                held = 0

    db = pd.DataFrame.from_dict(cap, orient='index')
    pct_above_sell_limit = (db['Gain'] > sell_limit).mean()
    pct_above_zero = (db['Gain'] > 0).mean()
    cap_gain = np.prod(db['Gain'] + 1) - 1
    return pct_above_sell_limit, pct_above_zero, cap_gain

In [105]:
start_dates = pd.date_range('2018-01-01', '2020-12-31', freq='QS')
end_dates = pd.date_range('2018-01-01', '2020-12-31', freq='Q')
dic = {}
cols = ['Above Sell Limit', 'Above Zero', 'Cap Gain']
for ticker in tickers:
    stat = []
    try:
        for start, end in zip(start_dates, end_dates):
            data = DataLoader(ticker).get_data(start.strftime('%Y-%m-%d'), 
                                            end.strftime('%Y-%m-%d'))
            data['Previous_Close'] = data['Close'].shift(1)
            scores = sim(data)
            stat.append(scores)
        avg_quarterly_status = np.array(stat).mean(0)
    except Exception as e:
        continue
    dic[ticker] = {cols[i]: aqs for i, aqs in enumerate(avg_quarterly_status)}

In [106]:
df = pd.DataFrame.from_dict(dic, orient='index')

In [107]:
mask = df['Cap Gain'].between(.6, 10)
backtest_tickers = df[mask].index.to_list()
backtest_tickers

['MVIS',
 'ATOS',
 'CRIS',
 'UAVS',
 'AWH',
 'VTGN',
 'PRTG',
 'IBIO',
 'HGEN',
 'GNUS',
 'BTX',
 'MMAT']

In [114]:
start_dates = pd.date_range('2021-01-01', '2021-12-31', freq='QS')
end_dates = pd.date_range('2021-01-01', '2021-12-31', freq='Q')
dic = {}
for ticker in backtest_tickers:
    stat = []
    try:
        for start, end in zip(start_dates, end_dates):
            data = DataLoader(ticker).get_data(start.strftime('%Y-%m-%d'), 
                                            end.strftime('%Y-%m-%d'))
            data['Previous_Close'] = data['Close'].shift(1)
            scores = sim(data)
            stat.append(scores)
            if scores[-1] < .2:
                break
    except Exception as e:
        continue
    total_status = np.array(stat)
    dic[ticker] = {'Above Sell Limit': total_status.mean(0)[0],
                   'Above Zero': total_status.mean(0)[1],
                   'Cap Gain': total_status.sum(0)[-1]}

In [115]:
df = pd.DataFrame.from_dict(dic, orient='index')

In [116]:
df

,Above Sell Limit,Above Zero,Cap Gain
MVIS,0.338095,0.900000,2.088677
ATOS,0.362418,0.902614,2.180882
CRIS,0.416667,0.833333,0.012357
UAVS,0.447964,0.893665,0.257283
AWH,0.335165,0.708791,-0.004273
VTGN,0.271062,0.801740,0.919798
PRTG,0.285714,0.857143,0.140650
IBIO,0.233333,0.685714,0.530141
HGEN,0.303846,0.861538,0.421940
GNUS,0.125000,0.769697,1.678272
